In [ ]:
from pathlib import Path
import datetime

import talib
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import lightgbm as lgb
from sklearn import svm, neural_network

import crypto
import data_fetcher

In [ ]:
def get_df(symbol, start_date, end_date, interval):
    fetcher = data_fetcher.gmo.GMOFethcer()
    df = fetcher.fetch_ohlc(
        symbol, interval=datetime.timedelta(minutes=interval), start_date=start_date, end_date=end_date
    )
    df = crypto.features.calc_features(df)
    df = df.filter(
        pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
    )
    return df

def calc_profits(df, suffix):
    df = df.with_columns(
        pl.Series(
            crypto.simulate.simulate_long_trade(
                df, f"sell_executed{suffix}", f"buy_executed{suffix}", f"target_price{suffix}", f"target_price{suffix}", wall_timestep=100
            )
        ).alias(f"profits{suffix}")
    )    
    return df

In [ ]:
symbol = "BTC_JPY"
interval = 1
start_date = datetime.datetime(2024, 1, 1)
end_date = start_date + datetime.timedelta(days=60)
df = get_df(symbol, start_date=start_date, end_date=end_date, interval=interval)

In [ ]:
# 決まった値幅で利確・損切りする戦略
range_rate = 0.2
wall_timestep = 10
suffix = "_atr"
df = df.with_columns(
    (pl.col("close") + pl.col("ATR") * range_rate).shift().alias(f"target_high_price{suffix}"),
    (pl.col("close") - pl.col("ATR") * range_rate).shift().alias(f"target_low_price{suffix}"),
    (pl.col("close") + pl.col("ATR") * range_rate * 2).shift().alias(f"losscut_high_price{suffix}"),
    (pl.col("close") - pl.col("ATR") * range_rate * 2).shift().alias(f"losscut_low_price{suffix}"),
    (pl.col("close").shift() < pl.col("low")).alias(f"buy_executed{suffix}"),
    ((pl.col("high") - pl.col("close")) < (pl.col("close") - pl.col("low"))).alias("near_high"),
    (pl.col("close").shift() > pl.col("high")).alias(f"sell_executed{suffix}"),
    (pl.col("close").rolling_max(window_size=10) == pl.col("close")).alias("rolling_max"),
    (pl.col("close").rolling_min(window_size=10) == pl.col("close")).alias("rolling_min"),
)

profits_buy = np.zeros(len(df))
profits_sell = np.zeros(len(df))
for i in range(1, len(df) - wall_timestep): #len(df) - wall_timestep):
    if df["near_high"][i] and df["near_high"][i - 1] and df["rolling_max"][i]:
        #if df[f"buy_executed{suffix}"][i + 1] and df[f"near_high"][i + 1]:
        buy_price = df["close"][i + 1]
        sell_price = df["close"][i + wall_timestep]
        profits_buy[i] = (sell_price - buy_price) / buy_price

    if not df["near_high"][i] and not df["near_high"][i - 1] and df["rolling_min"][i]:
        #if df[f"sell_executed{suffix}"][i + 1]:
        sell_price = df["close"][i + 1]
        buy_price = df["close"][i + wall_timestep]
        profits_sell[i] = (sell_price - buy_price) / buy_price
        #print(i, sell_price, buy_price, profits_sell[i])

In [ ]:
start_idx = 0
end_idx = len(df)
plt.plot(profits_buy.cumsum()[start_idx:end_idx], label="profits_buy")
plt.plot(profits_sell.cumsum()[start_idx:end_idx], label="profits_sell")
plt.plot((df["close"] / df["close"][0] - 1.0)[start_idx:end_idx], label="close")
plt.legend()
plt.grid()

In [ ]:
# volume barで試してみる
fetcher = data_fetcher.gmo.GMOFethcer()
vb_df = fetcher.fetch_volume_bar(symbol, volume_size=0.5, start_date=start_date, end_date=end_date)
vb_df = crypto.features.calc_features(vb_df)
vb_df = vb_df.filter(
    pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
)

In [ ]:
# 決まった値幅で利確・損切りする戦略
def calc_algorighm(df):
    range_rate = 0.2
    wall_timestep = 10
    suffix = "_atr"
    df = df.with_columns(
        (pl.col("close") + pl.col("ATR") * range_rate).shift().alias(f"target_high_price{suffix}"),
        (pl.col("close") - pl.col("ATR") * range_rate).shift().alias(f"target_low_price{suffix}"),
        (pl.col("close") + pl.col("ATR") * range_rate * 2).shift().alias(f"losscut_high_price{suffix}"),
        (pl.col("close") - pl.col("ATR") * range_rate * 2).shift().alias(f"losscut_low_price{suffix}"),
        (pl.col("close").shift() < pl.col("low")).alias(f"buy_executed{suffix}"),
        ((pl.col("high") - pl.col("close")) < (pl.col("close") - pl.col("low"))).alias("near_high"),
        (pl.col("close").shift() > pl.col("high")).alias(f"sell_executed{suffix}"),
        (pl.col("close").rolling_max(window_size=10) == pl.col("close")).alias("rolling_max"),
        (pl.col("close").rolling_min(window_size=10) == pl.col("close")).alias("rolling_min"),
    )

    profits_buy = np.zeros(len(df))
    profits_sell = np.zeros(len(df))
    for i in range(1, len(df) - wall_timestep): #len(df) - wall_timestep):
        if df["near_high"][i] and df["near_high"][i - 1] and df["rolling_max"][i]:
            #if df[f"buy_executed{suffix}"][i + 1] and df[f"near_high"][i + 1]:
            buy_price = df["close"][i + 1]
            sell_price = df["close"][i + wall_timestep]
            profits_buy[i] = (sell_price - buy_price) / buy_price

        if not df["near_high"][i] and not df["near_high"][i - 1] and df["rolling_min"][i]:
            #if df[f"sell_executed{suffix}"][i + 1]:
            sell_price = df["close"][i + 1]
            buy_price = df["close"][i + wall_timestep]
            profits_sell[i] = (sell_price - buy_price) / buy_price
    return profits_buy, profits_sell

In [ ]:
profits_buy, profits_sell = calc_algorighm(vb_df)

In [ ]:
plt.plot(profits_buy.cumsum(), label="profits_buy")
plt.plot(profits_sell.cumsum(), label="profits_sell")
plt.plot(vb_df["close"] / vb_df["close"][0] - 1.0, label="close")
plt.grid()
plt.legend()

In [ ]:
(vb_df["start_date"][1:] - vb_df["start_date"][:-1]).mean()